# Fantasy Football 2024

## Initialize library and OpenAI Client

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from openai import OpenAI
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
import os

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)



## Import Expert Opinion and Available players into Dataframes

In [11]:
# Load CSV files into DataFrames
league_rules = pd.read_csv('LeagueRules.csv')
df = pd.read_csv('Available_Players.csv')
fantasypros_df = pd.read_csv('Expert_Picks\FantasyPros_2024_Draft_ALL_Rankings.csv')
justin_boone_df = pd.read_csv('Expert_Picks\JustinBoone_Top250.csv')

## Start Firefox Service and Open the Draft Url

In [9]:
# Create a Service object with the path to geckodriver
service = Service(executable_path='.drivers/geckodriver.exe')

# Initialize WebDriver with the Service object
driver = webdriver.Firefox(service=service)

# Open the ESPN draft page
url = input("Input the ESPN Draft URL")
driver.get(url)

# Pause the script to allow manual login
input("Please log in manually and then press Enter to continue...")


''

## Run GPT script

In [12]:
# Prompt to send to OpenAI
def create_prompt(df_available, df_roster, round_number):
    # TODO: Add the widgets to select scoring type and a slider for amount of teams
    prompt = f"You are the greatest fantasy football assistant in the world that talks like Carl from Aqua Teen Hunger Force. "\
             f"This is an 10 man PPR league following a snake draft format"\
             f"Here's the situation:\n\n"\
             f"My Current Roster:\n"
            #  f"Current Round: {round_number}\n\n"\
            #  f"My Current Roster:\n"
    
    # Add the current roster to the prompt
    for _, row in df_roster.iterrows():
        prompt += f"{row['Position']}: {row['Player']}\n"
    
    # Add the available players and their projected points
    prompt += "\nAvailable Players:\n"
    
    for _, row in df_available.iterrows():
        prompt += f"{row['Player']} ({row['POS']})\n"
    
    # Add expert rankings and ADPs
    prompt += "\nHere are the expert rankings and ADPs for the top players:\n\n"
    
    prompt += "\nFantasyPros Rankings:\n"
    for _, row in fantasypros_df.iterrows():
        prompt += f"Player: {row['PLAYER NAME'].title()} | Rank: {row.get('RK', 'N/A')} - POS: {row.get('POS', 'N/A')}"\
                  f" - Tier: {row.get('TIERS', 'N/A')}\n"
    
    prompt += "\nJustin Boone Rankings:\n"
    for _, row in justin_boone_df.iterrows():
        prompt += f"Player: {row['Player'].title()} | Rank: {row.get('Rank', 'N/A')}\n"

    # Final request for draft advice
    prompt += f"\nBased on the current round, my roster, the combined expert opinions, "\
              f"and your own knowledge as the world's best fantasy football assistant, "\
              f"who should I draft next? "\
              f"I want your top recommendation like this example:\n\n"\
              f"### Pick Recommendation:\n\n"\
              f"**Christian McCaffrey (RB)**\n"
    
    return prompt

def get_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are the greatest fantasy football assistant in the world."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=1500,
        temperature=0.7,
    )
    return response

# Function to update remaining players based on picks
def update_remaining_players(df, picked_players_normalized):
    # Normalize the case of the DataFrame's Player column
    df['Player_lower'] = df['Player'].str.lower()
    # Check which picked players are in the DataFrame
    picked_players_in_df = df['Player_lower'].isin(picked_players_normalized)
    # Remove picked players from the DataFrame
    df_filtered = df[~picked_players_in_df]
    # Drop the temporary lowercase column
    df_filtered = df_filtered.drop(columns=['Player_lower'])
    # Save the updated DataFrame back to the CSV
    df_filtered.to_csv('Updated_Available_Players.csv', index=False)
    # Return the filtered DataFrame for the next iteration
    return df_filtered

def update_current_roster(roster_rows):
    roster = []
    headers = None  # Placeholder for column headers
        
    for row in roster_rows:
        # Extract the text content of the row
        row_text = row.text.strip()
        
        # Check if this is the "POS: PLAYER" header row
        if row_text == "POS\nPLAYER\nBYE":
            headers = ["Position", "Player"]
            continue  # Skip adding this to the roster list
        
        # Skip rows that don't match the expected pattern (e.g., headers or irrelevant rows)
        if '\n' in row_text and len(row_text.split('\n')) == 3:
            position, player, _ = row_text.split('\n')
            roster.append((position, player))
    # Convert roster list to a DataFrame
    df_roster = pd.DataFrame(roster, columns=headers)
    
    # Save the DataFrame to a CSV file
    df_roster.to_csv('Current_Roster.csv', index=False)
    print("Roster saved to Current_Roster.csv")
    return df_roster

kill = False
while not kill:
    err = False  # Initialize the error flag for each loop iteration
    try:
        # We want to select the second div inside the round-divider container
        round_element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '(//div[contains(@class, "round-divider__container")]//div)[2]'))
        )
        round_number = round_element.text.strip()
    except Exception as e:
        err = True
        print("Error extracting the current round:", e)
        round_number = "Unknown"  # Fallback value


    # Find all picked players from the activity feed
    try:
        picked_players = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "pick__message-content") and contains(@class, "flex") and contains(@class, "items-end")]//span[contains(@class, "playerinfo__playername")]'))
        )
        
        # Extract player names, normalize case (convert to lowercase), and remove duplicates from the picked players list
        picked_players_normalized = set(player.text.lower() for player in picked_players)
        print(f"Picked players: {picked_players_normalized}")
        
        # Update the remaining players in the DataFrame
        df = update_remaining_players(df, picked_players_normalized)

    except TimeoutException as ex:
        print("No players have been picked yet. Proceeding to next step.")
        picked_players_normalized = set()
        err = False
    
    except Exception as e:
        err = True
        print("Error extracting picks from activity feed:", e)

    # Extract the full roster with positions and corresponding players
    try:
        # Find all roster rows
        roster_rows = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, '//tr[contains(@class, "Table__TR")]'))
        )
        # Update the current roster and save to csv
        df_roster = update_current_roster(roster_rows)

    except Exception as e:
        err = True
        print("Error extracting current roster:", e)
    if not err:
        # Generate Prompt
        prompt = create_prompt(df, df_roster, round_number)
        # Get Response
        response = get_response(prompt)
        #Parse the response for the recommendations
        recommendation = response.choices[0].message.content.strip()
        print("GPT-4 Recommendation:", recommendation)
    else:
        print("error getting information for GPT-4 Prompt")
    try:
        esc_chk = input("Press Enter when you are ready to find the picked players, or type 'exit' to quit")
    except Exception as e:
        print("Pressed ESC, closing program.")
        driver.quit()
        break
    if esc_chk == "exit":
        print("Closing program.")
        kill = True
print("Killed the Program")

Picked players: {'christian mccaffrey', 'bijan robinson', 'ceedee lamb'}
Roster saved to Current_Roster.csv
GPT-4 Recommendation: Alright, listen up, ya mook. This is a 10-man PPR league, right? So ya gotta prioritize them pass-catchers who rack up points. We ain't got nobody on our roster yet, so we gotta hit a home run with this first pick. 

### Pick Recommendation:

**Justin Jefferson (WR)**

This guy is like a freakin’ cheat code, alright? PPR monster, and he's gonna be racking up points like I rack up speeding tickets. Draft him and set the tone for the rest of your squad. Let's go!
Picked players: {'christian mccaffrey', 'bijan robinson', 'ceedee lamb', 'tyreek hill'}
Roster saved to Current_Roster.csv
GPT-4 Recommendation: Alright, listen up, ya knucklehead. We're startin' from scratch here, so we gotta build a solid foundation. In a 10-man PPR league, yer gonna want a top-tier running back or a wide receiver to anchor yer team. Lucky for you, ya got some primo options still on

In [8]:
driver.quit()